In [1]:
import win32com.client

xl = win32com.client.Dispatch("Excel.Application")  #instantiate excel app

wb = xl.Workbooks.Open(r'C:\Users\USER\OneDrive - COMPANY\Merchandising Documents\AX Imports\Store\Sections\Execute\SectionsUploadExecute.xlsb')
# xl.Application.Run('macro.xlsm!Module1.macro1("Jay")')

xl.Application.Run('SectionsUploadExecute.xlsb!mod_GenerateSectionsUploadFile.CreateSectionsUploadFile')

# wb.Save()

xl.Application.Quit()

print('Done.')

Done.


In [2]:
import warnings
from datetime import datetime
import pyodbc
import pandas as pd
import os
import glob
import numpy as np
import shutil

warnings.filterwarnings('ignore')

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

query = '''

SELECT *

FROM InventItemBarCode

'''

dfItems = pd.read_sql(query, cnxn)

fmask = 'C:/Users/USER/OneDrive - COMPANY/Merchandising Documents/AX Imports/Store/Sections/*.xlsx'

# lst of column names which needs to be string
lst_str_cols = ['ItemID', 'StoreID']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}
# use dict on dtypes
# pd.read_csv('sample.csv', dtype=dict_dtypes)

dfs = []
for f in glob.glob(fmask):
    df = pd.read_excel(f, dtype=dict_dtypes)
    dfs.append(df)

# concatenated
dfSections = pd.concat(dfs, ignore_index=True)

dfSections = dfSections.rename({'ItemID': 'BarCode'}, axis = 1)

from functools import reduce

data_frames = [dfSections, dfItems]

# df_merged = reduce(lambda left,right: pd.merge(left,right,on=['BarCode'], how='left'), data_frames).fillna('')

df_merged = reduce(lambda left,right: pd.merge(left,right,left_on=['BarCode'],right_on=['ITEMBARCODE'], how='left'), data_frames).fillna('null')

df_merged.drop(df_merged.columns[[4,6,7,8,9,10]], axis = 1, inplace = True)

df_merged = df_merged.reindex(columns=['ITEMID', 'BarCode', 'SectionID', 'StoreID', 'ShelfID'])

#df_merged.to_excel('2022_07_21 RetailSections.xlsx', index=False, header=True)

numOfRows = len(dfSections.index)
print('Total number of rows in dataframe: ' , numOfRows)

# conditional column

df_merged['ITEMID2'] = np.where(((df_merged['ITEMID'] == 'null') & (df_merged['BarCode'].str.startswith('1000') == True)), df_merged['BarCode'], df_merged['ITEMID'])

df_merged['ITEMID2'].mask(df_merged['ITEMID2'] == 'null', '', inplace=True)

df_merged.drop(['ITEMID'], axis = 1, inplace = True)

cols = df_merged.columns.tolist()

cols = cols[-1:] + cols[:-1]

df_merged = df_merged[cols]

df_merged.rename(columns={'ITEMID2': 'ITEMID'}, inplace=True)

bl = df_merged[df_merged['ITEMID'] == '']

print('There are some blank records.')

df_merged.drop(['BarCode'], axis = 1, inplace = True)

# dropping ALL duplicate values
df_merged.drop_duplicates(subset = ['ITEMID', 'SectionID'], keep = False, inplace = True)

myPath = 'C:/Users/USER/OneDrive - COMPANY/Merchandising Documents/AX Imports/Store/Sections/Output/'

myDate = pd.datetime.today().strftime('%Y_%m_%d %H_%M')

if not bl.empty:
    bl.to_excel(myPath + myDate + ' RetailSections Blanks.xlsx', index=False, header=True)
    df_merged['ITEMID'].replace('', np.nan, inplace=True)
    df_merged.dropna(subset=['ITEMID'], inplace=True)

# myDate = pd.datetime.today().strftime('%Y_%m_%d')

df_merged.to_excel(myPath + myDate + ' RetailSections.xlsx', index=False, header=True)

myPath = 'C:\\Users\USER\\OneDrive - COMPANY\\Merchandising Documents\\AX Imports\\Store\\Sections\\Output\\'

file_to_copy = myPath + myDate + ' RetailSections.xlsx'

myPath = '\\\\is3-01\\aximport\\RetailInventItemSectionLocation\\Work\\'

file_target = myPath + myDate + ' RetailSections.xlsx'

shutil.copyfile(file_to_copy, file_target)
    
print('Done.')

Total number of rows in dataframe:  1602
There are some blank records.
Done.
